In [1]:
import random
import heapq
from dataclasses import dataclass, field
from typing import List, Tuple, Callable, Any, Set
import numpy as np
from time import time
from tabulate import tabulate
import threading
import collections
from math import ceil
from enum import Enum
from functools import partial
from copy import deepcopy

import ctypes as ct 

In [2]:

class LightsOutPuzzle:
    def __init__(self, board: List[List[int]]):
        self.board = np.array(board, dtype=np.uint8)
        self.size = len(board)

    def toggle(self, x, y):
        for dx, dy in [(0, 0), (1, 0), (-1, 0), (0, 1), (0, -1)]:
            nx, ny = x + dx, y + dy
            if 0 <= nx < self.size and 0 <= ny < self.size:
                self.board[nx, ny] ^= 1

    def is_solved(self):
        return np.all(self.board == 0)

    def get_moves(self):
        return [(x, y) for x in range(self.size) for y in range(self.size)]

    def __str__(self):
        return '\n'.join(' '.join(str(cell) for cell in row) for row in self.board)


In [3]:
def create_random_board(size: int,  seed: int, num_toggles: int = None):
    random.seed(time() if seed is None else seed)

    board = [[0 for _ in range(size)] for _ in range(size)]
    puzzle = LightsOutPuzzle(board)

    if num_toggles is None:
        num_toggles = random.randint(1, size * size)

    for _ in range(num_toggles):
        x = random.randint(0, size - 1)
        y = random.randint(0, size - 1)
        puzzle.toggle(x, y)

    return puzzle.board

In [4]:
def show_solution(
    puzzle: LightsOutPuzzle,
    solution: List[Tuple[int, int]],
    algorithm_name: str,
    nodes_visited: int,
    show_steps: bool = False,
):
    print(f"\nSolving with {algorithm_name}:")
    if solution is None:
        print("No solution found.")
        return
    print(f"Solution: {solution}")
    print(f"Nodes visited: {nodes_visited}")
    if show_steps:
        print("\nSolution steps:")
        for i, move in enumerate(solution):
            print(f"\nStep {i+1}: Toggle position {move}")
            puzzle.toggle(*move)
            print(puzzle)
        print(
            "\nFinal state (solved):"
            if puzzle.is_solved()
            else "\nFinal state (not solved):"
        )
        print(puzzle)

In [5]:
tests = [
    create_random_board(3, 42),
    create_random_board(3, 41),
    create_random_board(3, 42, 5),
    create_random_board(4, 42),
    create_random_board(4, 41),
    create_random_board(4, 42, 5),
    create_random_board(5, 42),
    create_random_board(5, 41),
    create_random_board(5, 42, 5),
]

In [6]:
# TODO: Must return a list of tuples and the number of visited nodes
def bfs_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    visited = []
    curr_board=[]
    visited_nums=0
    bfs_queue = ()
    size=puzzle.size
    main_state=puzzle
    curr_state=deepcopy(main_state)
    curr_board.append((curr_state.board).tolist())
    curr_path=()
    visited.append(curr_board)
    bfs_queue=bfs_queue+((curr_board[0], [-1,-1]), )
    if(size >= 5):
        return None, 0
    
    while (bfs_queue):
        curr_state=deepcopy(main_state)
        curr_path=curr_path + (bfs_queue[0][1:])
        for (the_x, the_y) in curr_path:
            curr_state.toggle(the_x, the_y)
            visited_nums= visited_nums+1
        if curr_state.is_solved():
            return curr_path[1:], visited_nums
        
        for i in range (size):
            for j in range(size):
                curr_board=[]
                curr_state.toggle(i , j)
                curr_board.append((curr_state.board).tolist())
                
                if curr_board not in visited:
                    the_path= curr_path+([i,j],)
                    board_tuple=(curr_board[0], )
                    bfs_queue= bfs_queue+ ((board_tuple + the_path[0:]),)
                    visited.append(curr_board)
                    
                curr_state.toggle(i,j)
        bfs_queue= bfs_queue[1:]
        curr_path=()
    return None, visited_nums
              


In [7]:
def DLS(the_state , depth , visited_num_ptr, path, size) -> Tuple[List[Tuple[int, int]], int]:
    curr_board=[]
    visited=[]
    new_state = deepcopy(the_state)
    visited.append(new_state)
    visited_num_ptr.contents.value +=1
    if the_state.is_solved():
        return path, visited_num_ptr.contents.value
    
    if depth <= 0:
        return None, visited_num_ptr.contents.value
    
    for i in range(size):
        for j in range(size):
            new_state.toggle(i, j)
            curr_board.append((new_state.board).tolist())
            if curr_board not in visited:
                new_path = path + ([i, j],)
                result = DLS(new_state, depth-1, visited_num_ptr, new_path, size)
            if result[0] is not None:
                return result
            new_state.toggle(i,j)
    return None , visited_num_ptr.contents.value

In [8]:
# TODO: Must return a list of tuples and the number of visited nodes
def ids_solve(puzzle: LightsOutPuzzle) -> Tuple[List[Tuple[int, int]], int]:
    
    size=puzzle.size
    main_state=puzzle
    depth = 0
    visited_nums = ct.c_long(0)
    visited_nums_ptr = ct.pointer(visited_nums) 
    
    if(size >= 5):
        return None, 0
    
    while True:
        curr_state=deepcopy(main_state)
        curr_path = ()
        result = DLS(curr_state, depth, visited_nums_ptr,curr_path, size)
        if result[0] is not None:
            return result
        depth += 1
    
    return None, visited_nums.value

In [9]:
# TODO: Must return a list of tuples and the number of visited nodes
def astar_solve(puzzle: LightsOutPuzzle, heuristic: Callable[[LightsOutPuzzle], int]) -> Tuple[List[Tuple[int, int]], int]:
    visited =set()
    visited_nums= 0
    fringe =[]
    size= puzzle.size
    main_state= puzzle
    curr_state=deepcopy(main_state)
    initial_board=tuple(map(tuple, curr_state.board.tolist()))
    curr_path= [(-1,-1)]
    visited.add((initial_board, 0))
    heapq.heappush(fringe, (0, initial_board, curr_path))
    
    
    
    while fringe:
        f_value, curr_board, curr_path=heapq.heappop(fringe)
        curr_state=deepcopy(main_state)
        for (the_x, the_y) in curr_path:
            curr_state.toggle(the_x, the_y)
        if curr_state.is_solved():
            return curr_path[1:], visited_nums

        for i in range(size):
            for j in range(size):
                curr_state.toggle(i, j)
                new_f= len(curr_path) + heuristic(curr_state)
                new_board= tuple(map(tuple, curr_state.board.tolist()))
                board_tuple= (new_board, new_f)

                if board_tuple not in visited:
                    the_path= curr_path+[(i, j)]
                    heapq.heappush(fringe, (new_f, new_board, the_path))
                    visited.add(board_tuple)
                    visited_nums +=1
                    
                curr_state.toggle(i, j)  
                
                
    return None, visited_nums




In [10]:
# TODO: Implement heuristic functions and store them in the list
def heuristic1(puzzle: LightsOutPuzzle):
  return np.sum(np.array(puzzle.board))

def heuristic2(puzzle: LightsOutPuzzle) :
  return (np.sum(np.array(puzzle.board))**2)

def heuristic3(puzzle: LightsOutPuzzle):
  size=puzzle.size
  the_board=puzzle.board
  count = 0
  for i in range (size):
    for j in range (size):
      if (the_board[i][j]==1):
        count +=1
        if i > 0 and the_board[i-1][j] == 0:
          count +=1
        if i < size-1 and the_board[i+1][j] == 0:
          count +=1
        if j > 0 and the_board[i][j-1] == 0:
          count +=1
        if j < size-1 and the_board[i][j+1] == 0:
          count +=1
  return count


heuristics = [heuristic1, heuristic2, heuristic3]

In [11]:
# TODO: Implement weighted heuristic functions and store them in the list
def weighted_heuristic1(puzzle: LightsOutPuzzle, alpha):
  return heuristic1(puzzle) * alpha
def weighted_heuristic2(puzzle: LightsOutPuzzle, alpha):
  return heuristic2(puzzle) * alpha
def weighted_heuristic3(puzzle: LightsOutPuzzle, alpha):
  return heuristic3(puzzle) * alpha

weighted_heuristics = [weighted_heuristic1, weighted_heuristic2, weighted_heuristic3]

# TODO: assign 2 weights you wanna run
weights = [1.3, 1.6]

In [12]:
class SearchStatus(Enum):
    SOLVED = "Solved"
    TIMEOUT = "Timeout"
    NO_SOLUTION = "No Solution"

@dataclass
class SearchResult:
    result: SearchStatus
    solution: Any = None
    nodes_visited: int = None
    time: float = None

    def __str__(self):
        output = f"Result: {self.result.value}\n"
        if self.result == SearchStatus.SOLVED:
            output += f"Solution: {self.solution}\n"
        if self.result != SearchStatus.TIMEOUT:
            output += f"Nodes Visited: {self.nodes_visited}\n"
            output += f"Time Taken: {self.time:.3f} seconds\n"

        return output


DEFAULT_SEARCH_RESULT = SearchResult(SearchStatus.TIMEOUT, None, None, float("inf"))

In [13]:
def run_searches(func: Callable, args: Any, time_limit: float = 60.0, name: str = "") -> SearchResult:
    result = []

    def target():
        try:
            solution, nodes_visited = func(*args)
            result.append((solution, nodes_visited))
        except Exception as e:
            result.append(e)

    thread = threading.Thread(target=target)
    thread.start()

    start_time = time()
    thread.join(timeout=time_limit)

    if thread.is_alive() or not result:
        print(f"\nTime limit of {time_limit} seconds exceeded for {name}")
        return SearchResult(SearchStatus.TIMEOUT)

    if isinstance(result[0], Exception):
        raise result[0]

    solution, nodes_visited = result[0]
    show_solution(args[0], solution, name, nodes_visited)
    time_taken = time() - start_time
    return SearchResult(SearchStatus.SOLVED if solution else SearchStatus.NO_SOLUTION, solution, nodes_visited, time_taken)

In [14]:
def run_test(test: List, heuristics: List[Callable[[Any], int]], time_limit: float = 60.0):

    print(f"Running test:\n {test}")

    puzzle = LightsOutPuzzle(test)

    bfs_result = run_searches(bfs_solve, (deepcopy(puzzle),), name="BFS", time_limit=time_limit)
    ids_result = run_searches(ids_solve, (deepcopy(puzzle),), name="IDS", time_limit=time_limit)

    astar_results = []
    for heuristic in heuristics:
        astar_results.append(
            run_searches(
                astar_solve,
                (deepcopy(puzzle), heuristic),
                name=f"A*({heuristic.__name__})",
                time_limit=time_limit,
            )
        )

    weighted_astar_results = []
    for heuristic in weighted_heuristics:
        for weight in weights:
          weighted_astar_results.append(
              run_searches(
                  astar_solve,
                  (deepcopy(puzzle), partial(heuristic, alpha = weight)),
                  name=f"weighted A*({heuristic.__name__}, alpha = {weight})",
                  time_limit=time_limit,
              )
          )

    print("\n-------------------------------\n")

    return (
        test,
        bfs_result,
        ids_result,
        *astar_results,
        *weighted_astar_results
    )

In [15]:
results: List[Tuple[str, SearchResult, SearchResult, SearchResult, SearchResult]] = []

for test in tests:
    results.append(run_test(test, heuristics, 180 ))

Running test:
 [[1 1 1]
 [1 1 1]
 [1 0 0]]

Solving with BFS:
Solution: ([0, 2], [1, 0])
Nodes visited: 67

Solving with IDS:
Solution: ([0, 2], [1, 0])
Nodes visited: 37

Solving with A*(heuristic1):
Solution: [(1, 0), (0, 2)]
Nodes visited: 43

Solving with A*(heuristic2):
Solution: [(1, 0), (0, 2)]
Nodes visited: 79

Solving with A*(heuristic3):
Solution: [(1, 0), (0, 2)]
Nodes visited: 18

Solving with weighted A*(weighted_heuristic1, alpha = 1.3):
Solution: [(1, 0), (0, 2)]
Nodes visited: 43

Solving with weighted A*(weighted_heuristic1, alpha = 1.6):
Solution: [(1, 0), (0, 2)]
Nodes visited: 43

Solving with weighted A*(weighted_heuristic2, alpha = 1.3):
Solution: [(1, 0), (0, 2)]
Nodes visited: 79

Solving with weighted A*(weighted_heuristic2, alpha = 1.6):
Solution: [(1, 0), (0, 2)]
Nodes visited: 97

Solving with weighted A*(weighted_heuristic3, alpha = 1.3):
Solution: [(1, 0), (0, 2)]
Nodes visited: 18

Solving with weighted A*(weighted_heuristic3, alpha = 1.6):
Solution: [(1

In [16]:
table_data: List[List[str]] = []

for result in results:
    table_data.append(
        [
            "\n".join(" ".join(str(cell) for cell in row) for row in result[0]),
            str(result[1]),
            str(result[2]),
        ]
    )

    for i in range(len(heuristics)):
        table_data[-1].append(str(result[3 + i]))

    for i in range(len(weighted_heuristics)):
        for j in range(len(weights)):
          table_data[-1].append(str(result[3 + len(heuristics) + i * 2 + j]))

    table_data.append("\n\n\n")

if table_data[-1] == "\n\n\n":
    table_data.pop()

print(len(table_data))
print("Results:")
print(
    tabulate(
        table_data,
        headers=[
            "Test",
            "BFS",
            "IDS",
            *[f"A* ({heuristic.__name__})" for heuristic in heuristics],
            *[f"Weighted A* ({heuristic.__name__}, alpha = {weight})" for heuristic in weighted_heuristics for weight in weights]
        ],
        floatfmt=".3f",
        numalign="center",
        stralign="center",
        tablefmt="grid",
    )
)


17
Results:
+-----------+----------------------------------------------------+----------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------